In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.autograd import Function
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import qiskit
from qiskit.visualization import *
from qiskit import ClassicalRegister

In [7]:
QC_outputs = ['000', '001', '010', '011', '100', '101', '110', '111']

In [4]:
# Defines the quantum circuit so we can use it (since hybrids have quantum)
class QuantumCircuit:
  #This is the initialization
  def __init__(self, n_qubits, backend, shots):

    #Define how many wires/qubits we want
    self._circuit = qiskit.QuantumCircuit(n_qubits)

    # Just a list of 0 to the number of qubits... Just useful so we can just define the circuit (with all it's little parts super quickly)
    all_qubits = [i for i in range(n_qubits)]
    
    # Data dependent shifts
    self.val_rot0 = qiskit.circuit.Parameter('val_rot0')
    self.val_rot1 = qiskit.circuit.Parameter('val_rot1')
    self.val_rot2 = qiskit.circuit.Parameter('val_rot2')
    self.val_rot3 = qiskit.circuit.Parameter('val_rot3')


    # Parameterized shifts <- these are what we are training
    self.theta_0 = qiskit.circuit.Parameter('theta0')
    self.theta_1 = qiskit.circuit.Parameter('theta1')
    self.theta_2 = qiskit.circuit.Parameter('theta2')
    self.theta_3 = qiskit.circuit.Parameter('theta3')
    self.theta_4 = qiskit.circuit.Parameter('theta4')
    self.theta_5 = qiskit.circuit.Parameter('theta5')
    self.theta_6 = qiskit.circuit.Parameter('theta6')
    self.theta_7 = qiskit.circuit.Parameter('theta7')
    self.theta_8 = qiskit.circuit.Parameter('theta8')

    # Initilaize each qubit with a Hadamard
    self._circuit.h(all_qubits)
    self._circuit.barrier()

    # Apply the data dependent shifts
    self._circuit.ry(self.val_rot0, all_qubits)
    self._circuit.ry(self.val_rot1, all_qubits)
    self._circuit.ry(self.val_rot2, all_qubits)
    self._circuit.ry(self.val_rot3, all_qubits)
    self._circuit.barrier()

    #Now comes the custom thing for 3 qubits specifically. I adapted it from a paper
    self._circuit.cz(0,1)
    self._circuit.cz(1,2)
    self._circuit.cz(2,3)
    self._circuit.ry(self.theta_0, 0)
    self._circuit.ry(self.theta_1, 1)
    self._circuit.ry(self.theta_2, 2)
    self._circuit.ry(self.theta_3, 2)
    self._circuit.cz(0,1)
    self._circuit.cz(1,2)
    self._circuit.cz(2,3)
    self._circuit.ry(self.theta_4, 0)
    self._circuit.ry(self.theta_5, 1)
    self._circuit.ry(self.theta_6, 2)
    self._circuit.ry(self.theta_7, 2)

    self._circuit.measure_all()

    #save these varaibles for later so we don't have to call them again during the forwarding
    self.backend = backend
    self.shots = shots
  
  #forwarding through the quantum circuit
  def run(self, thetas):
    #prep the execution. Link to circuit, Define backend and number of shots... And then fill in the placeholder variables (theta) with the thing we pass through when we forward
    job = qiskit.execute(self._circuit,
                         self.backend,
                         shots = self.shots,
                         parameter_binds = [{self.theta_0: thetas[0],
                                             self.theta_1: thetas[1],
                                             self.theta_2: thetas[2],
                                             self.theta_3: thetas[3],
                                             self.theta_4: thetas[4],
                                             self.theta_5: thetas[5],
                                             self.theta_6: thetas[6],}])
    
    #execution
    counts = job.result().get_counts(self._circuit)

    print("Counts: ", counts)

    expects = np.zeros(8)
    for k in range(8):
      key = QC_outputs[k]
      perc = counts.get(key, 0) /self.shots
      expects[k] = perc
    return expects

In [5]:
#Going to test out the quantum circuit.
sim = qiskit.Aer.get_backend('qasm_simulator')

test_circuit = QuantumCircuit(3, sim, 100000)

In [6]:
#Heck yeah it works! It outputs the distributions of the outputs
test_circuit.run([1,2,1,4,5,6,8]), test_circuit._circuit.draw()

Counts:  {'000': 1795, '001': 2877, '010': 75891, '011': 6140, '100': 5136, '101': 2466, '110': 5476, '111': 219}


NameError: name 'QC_outputs' is not defined